In [1]:
import os
from os import listdir
from os.path import join,isfile
from tqdm import tqdm

import time
import copy

import cv2
import numpy as np
from tqdm import tqdm
import glob
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch import autograd
from torch.nn import functional as F
from torchvision import models


from misc_function import processImage, detail_enhance_lab, recreate_image, PreidictLabel, AdvLoss
from module import DeepGuidedFilter
from my_model import Model

In [4]:
def run(image, smooth_image, filename):
    # Adv img 폴더
    adv_path =    '../Adv_Img_mobilenet_v2/'
    if not os.path.isdir(adv_path):
        os.makedirs(adv_path)
        
    # 스무딩 loss
    criterion_L1 = nn.L1Loss()
    criterion_L2 = nn.MSELoss()
    guided_model = DeepGuidedFilter()
    optimizer = optim.Adam(guided_model.parameters(), lr=1e-3)

    with torch.cuda.device(0):
        guided_model.cuda()
        criterion_L1.cuda()
        criterion_L2.cuda()
    # Load 모델
    classifier = Model()
    classifier.load_state_dict(torch.load('../model_weight/mobilenet_v2.ckpt'))

    classifier.eval()
    classifier.cuda()

    # 모델 FC-layer 고정
    for param in classifier.parameters():
        param.requires_grad = False
        
    # 원본이미지, 스무딩이미지 전처리하고 tensor로 바꿔줌.
    x= processImage(image)    
    gt_smooth = processImage(smooth_image)
    # 정답 class, logit
    class_x, logit_x = PreidictLabel(x, classifier)

    #FCNN 최대 iter 설정
    maxIters = 5000

    for it in range(maxIters): 
        ori_label = torch.tensor(int(filename.split('_')[2][0])).cuda()
        if not torch.equal(ori_label, class_x):
            break
        '''
            gt_smooth : [11]로 스무딩된 이미지
            x_smooth : 뉴럴네트워크를 이용해서 스무딩 하는법을 배움.
        '''
        with autograd.detect_anomaly():
            # x를 guided filter에 넣고 smoothing 하는 방법을 배움. 
            x_smooth = guided_model(x,x)
            #디테일 강화
            enh,base = detail_enhance_lab(x,x_smooth)           
            # 디테일 강회 이미지 class, logit
            class_enh, logit_enh = PreidictLabel(enh.permute(2,0,1).unsqueeze(dim=0), classifier)
            
            #smoothing, adv loss
            loss1 = criterion_L2(x_smooth, gt_smooth)
            loss2 = criterion_L1(x_smooth, gt_smooth)
            loss3 = AdvLoss(logit_enh, class_x)
            
            # smoothing_loss, Adv_loss 비율 설정
            loss = 5*loss1  + loss3 + 5*loss2
            
            optimizer.zero_grad()
            loss.backward()
        
            torch.nn.utils.clip_grad_norm(guided_model.parameters(), 0.01)
            optimizer.step()
            check_enh = recreate_image(enh)
            check_enh = torch.from_numpy(np.flip(check_enh,axis=0).copy()).cuda()
            class_enh, _ = PreidictLabel(check_enh.permute(2,0,1).unsqueeze(dim=0), classifier)
            if it % 100 ==0:
                print(f'iter : {it} \t L2-loss:{loss1} \t L1-loss:{loss2} \t Adv_loss: {loss3}')
            if (class_x != class_enh): 
                cv2.imwrite('{}{}'.format(adv_path,filename), recreate_image(enh))
                if (loss1< 0.0005):
                    break


In [ ]:
image=[]
label=[]
filename=[]
for file_name in glob.glob('../Dataset/*.png'):
    filename.append(file_name.split('/')[2])
    label.append(file_name.split('_')[2][0])
    image.append(plt.imread(file_name)[...,:3])
image_set = list(map(lambda x: x, image))
for idx in tqdm(range(0,len(image_set))):
    smooth_image = plt.imread('../Smoothing_Imgs/'+filename[idx])
    run(image_set[idx],smooth_image,filename[idx])